# Python 利用Scikit-Learn套件實作TFIDF的方法
# 首先要先確定有 [字典] 和 [停用詞] 的txt檔

In [19]:
import os
import jieba
import jieba.analyse
import jieba.posseg as pseg  
jieba.set_dictionary('dict.big.tra.txt')  #預設字典
jieba.load_userdict('dict_twstd_tfidf.txt')  #中文分詞詞庫(TFIDF)
jieba.load_userdict('dict_ntusd_pos.txt') #NTUSD_負向
jieba.load_userdict('dict_ntusd_nag.txt') #NTUSD_正向
jieba.load_userdict('dict_twedu_dict.txt') #教育部《重編國語辭典 修訂本》


#jieba.load_userdict('dict_20160111_NotReallyUseful.txt')  
#中文分詞詞庫(TFIDF) + 百度分詞詞庫 + 69萬中文大辭典 + 中文热门词库(155073) 
#不是很好用，可備著使用


jieba.analyse.set_stop_words("stopwords_1227.txt") #停用詞

Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
DEBUG:jieba:Building prefix dict from C:\Users\BigData\dict.big.tra.txt ...
Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
DEBUG:jieba:Loading model from cache c:\users\bigdata\appdata\local\temp\jieba.ud8ec30fabaf60e161f06b3552aab2f0e.cache
Loading model cost 0.316 seconds.
DEBUG:jieba:Loading model cost 0.316 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


# 把斷詞結果存進陣列的方法
使用說明：

    wordsToList(傳入的檔名(TXT))

    檔名內容不拘，只要是在講同一個景點的都可以
    回傳會是檔案的斷詞結果，用List表示

In [20]:
def wordsToList(attfile):
    f = open(attfile,'r')
    f1 = f.read()
    f2 = ''.join(f1.split())
    #print f2
    words=jieba.cut(f2)
    n = ' '.join(words)
    n1 = n.encode('utf-8')
    return n1
    f.close()

# 使用停用詞的方法
使用說明：
    stopWordsList(傳入的檔名(TXT))
    
    傳入檔：停用詞的詞庫檔。內部格式必須為"一行一個詞語"( 詞語後不得有逗號 )
    使用後會把檔案內容，回傳成一個List

In [21]:
def stopWordsList(stopwordfile):
    li = []
    fid =  open(stopwordfile,'r')
    for line in fid:
        li.append(line.strip().decode('utf-8'))#要解開utf-8,才能成功給scikitlearn用
    fid.close()
    return li

List內容 比較UTF-8解開與否的結果

In [22]:
st = ['一個']
print st
st2 = ['一個'.decode('utf-8')]
print st2

['\xe4\xb8\x80\xe5\x80\x8b']
[u'\u4e00\u500b']


# 計算出不同文章間詞語TFIDF值的方法

使用方法：

    CompareTFIDF(listInput,stopWordsFile,rankNum)
    
    listInput：TXT檔，包含要傳入檔案名稱，合成的一個陣列
               例如：['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
               或是先新增一個陣列，再用變數去接住，傳變數也可以
               
    stopWordsFile：TXT檔，停用詞的詞庫檔。內部格式必須為"一行一個詞語"( 詞語後不得有逗號 )
    
    rankNum：數字(整數)，表示要列出各文章的排名前幾的詞語
    
使用範例：
    
    CompareTFIDF(['mountain.txt','lungshanTemple.txt','maokongGondola.txt'],'stopwords_1222.txt',5)
    
    or
    
    list = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
    CompareTFIDF(list,'stopwords_1222.txt',5)

In [23]:
def CompareTFIDF(listInput,stopWordsFile,rankNum):
    from sklearn import feature_extraction  
    from sklearn.feature_extraction.text import TfidfTransformer  
    from sklearn.feature_extraction.text import CountVectorizer
    
    #創造一個空矩陣
    corpus = []
    
    #把傳入的List依序讀出，使用方法把這些東西全都讀出來
    for each in range(0,len(listInput)):
        corpus.append(wordsToList(listInput[each]))

    vectorizer=CountVectorizer(analyzer='word',stop_words=stopWordsList(stopWordsFile))  #該類會將文本中的詞語轉換為詞頻矩陣，矩陣元素a[i][j] 表示j詞在i類文本下的詞頻  
    x = vectorizer.fit_transform(corpus)  #將文本轉為詞頻矩陣 
    transformer=TfidfTransformer()  #該類會統計每個詞語的tf-idf權值 
    tfidf=transformer.fit_transform(x)  #計算tf-idf
    word=vectorizer.get_feature_names()  #獲取詞袋模型中的所有詞語
    weight=tfidf.toarray()   #將tf-idf矩陣抽取出來，元素a[i][j]表示j詞在i類文本中的tf-idf權重  

    print '-------以下輸出各景點中TFIDF值最高的幾個詞語------\n'
    
    #打印每類文本的tf-idf詞語權重，第一個for遍歷所有文本，第二個for便利某一類文本下的詞語權重
    import operator
    for i in range(len(weight)):
        print "------景點 ",listInput[i]," 類文章內詞語tf-idf權重------" 
        print "排名",'\t',"詞語","\t","權重"
        print "- - - - - - - - - - - - - - - -"
        dic = {}
        for j in range(len(word)):  
            dic[word[j]] = weight[i][j]
        words_freq = sorted(dic.iteritems(),key=operator.itemgetter(1),reverse=True)
        num2 = 0
        for ele in words_freq[0:rankNum]:
            num2 += 1
            print num2,"\t",ele[0],"\t",ele[1]
        print '\n'

In [24]:
lis = ['mountain.txt','lungshanTemple.txt','maokongGondola.txt']
CompareTFIDF(lis,'stopwords_1227.txt',7)

-------以下輸出各景點中TFIDF值最高的幾個詞語------

------景點  mountain.txt  類文章內詞語tf-idf權重------
排名 	詞語 	權重
- - - - - - - - - - - - - - - -
1 	101 	0.419783594375
2 	象山 	0.348452306491
3 	地方 	0.234176798515
4 	城市 	0.213361083092
5 	大象 	0.192095502296
6 	值得 	0.168260366341
7 	夜景 	0.150913936821


------景點  lungshanTemple.txt  類文章內詞語tf-idf權重------
排名 	詞語 	權重
- - - - - - - - - - - - - - - -
1 	寺廟 	0.622179753309
2 	龍山寺 	0.488852121263
3 	地方 	0.191127113762
4 	臺灣 	0.142328701737
5 	夜市 	0.127418075841
6 	值得 	0.123351541506
7 	文化 	0.0977461960654


------景點  maokongGondola.txt  類文章內詞語tf-idf權重------
排名 	詞語 	權重
- - - - - - - - - - - - - - - -
1 	纜車 	0.516165487179
2 	貢多拉 	0.351399397095
3 	貓空 	0.33188958428
4 	動物園 	0.287509000335
5 	水晶 	0.260060925648
6 	等待 	0.125423389408
7 	玻璃 	0.119634617589


